# Progetto BigData
## Valerio Di Zio e Francesco Magnani

intro here

In [59]:
%%configure -f
{"executorMemory":"8G", "numExecutors":2, "executorCores":3, "conf": {"spark.dynamicAllocation.enabled": "false"}}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
3,application_1707323170996_0004,spark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
3,application_1707323170996_0004,spark,idle,Link,Link,None,✔


In [60]:
val bucketname = "unibo-bd2324-vdizio"

val path_book_sample = "s3a://"+bucketname+"/project/book_sample134.json"
val path_positive_words = "s3a://"+bucketname+"/project/positive-words.txt"
val path_negative_words = "s3a://"+bucketname+"/project/negative-words.txt"

"SPARK UI: Enable forwarding of port 20888 and connect to http://localhost:20888/proxy/" + sc.applicationId + "/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

bucketname: String = unibo-bd2324-vdizio
path_book_sample: String = s3a://unibo-bd2324-vdizio/project/book_sample134.json
path_positive_words: String = s3a://unibo-bd2324-vdizio/project/positive-words.txt
path_negative_words: String = s3a://unibo-bd2324-vdizio/project/negative-words.txt
res3: String = SPARK UI: Enable forwarding of port 20888 and connect to http://localhost:20888/proxy/application_1707323170996_0004/


In [92]:
case class BookReview(
    id: String,
    overall:Double,
    reviewTime:String,
    reviewerID:String,
    asin:String,
    reviewText:String,
    summary:String
)

object BookReview{
    def extract(row:org.apache.spark.sql.Row) = {
        val overall = row.getAs[Double]("overall")
        val reviewTime = row.getAs[String]("reviewTime")
        val reviewerID = row.getAs[String]("reviewerID")
        val asin = row.getAs[String]("asin")
        val reviewText = row.getAs[String]("reviewText")
        val summary = row.getAs[String]("summary")
        val id = reviewerID + "-" + asin
    
        new BookReview(id, overall, reviewTime, reviewerID, asin, reviewText, summary)
    }
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

defined class BookReview
defined object BookReview
Companions must be defined together; you may wish to use :paste mode for this.


In [93]:
val bookSampleRdd = spark.read.json(path_book_sample).map(BookReview.extract)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

bookSampleRdd: org.apache.spark.sql.Dataset[BookReview] = [id: string, overall: double ... 5 more fields]


In [94]:
bookSampleRdd.filter(_.overall<2).collect

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res33: Array[BookReview] =
Array(BookReview(A2M08SO0PJKPAV-0001712799,1.0,02 20, 2016,A2M08SO0PJKPAV,0001712799,Completly boring!!! Yes it's a childerns book that they will be able to read beacuse 60% of the book is the word Up.
This one never gets picked for story time just sits on the shelf.
We love Dr Seuss books but this one is disappointing.,Don't waste your money), BookReview(A15579Y1NDKSGW-0002005263,1.0,07 14, 2007,A15579Y1NDKSGW,0002005263,Characters you don't care about, exposition instead of action, Leaphorn and Chee MIA for much of the story...this one has it all.  I gave it a generous chance for five chapters, then called it quits because it was uninteresting and uninvolving.

Rob Schmidt
BlueCornComics.com,Worst Hillerman mystery ever?), BookReview(A1JYERR10WFM3O-000200526...


In [101]:
val positiveWords = sc.textFile(path_positive_words)
val negativeWords = sc.textFile(path_negative_words)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

positiveWords: org.apache.spark.rdd.RDD[String] = s3a://unibo-bd2324-vdizio/project/positive-words.txt MapPartitionsRDD[62] at textFile at <console>:24
negativeWords: org.apache.spark.rdd.RDD[String] = s3a://unibo-bd2324-vdizio/project/negative-words.txt MapPartitionsRDD[64] at textFile at <console>:24


In [102]:
def countOccurrences(src: String, tgt: String): Int =
  src.sliding(tgt.length).count(window => window == tgt)

def countPositive(text: String) = positiveWords.map(word => countOccurrences(text, word)).sum
def countNegative(text: String) = negativeWords.map(word => countOccurrences(text, word)).sum

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

countOccurrences: (src: String, tgt: String)Int
countPositive: (text: String)Double
countNegative: (text: String)Double


In [103]:
bookSampleRdd.
    map(review => (review.id, review.reviewText)).
    filter(_._1 != null).
    map({case (id, line) => (id, countPositive(line) + countNegative(line))}).collect

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
org.apache.spark.SparkException: Job aborted due to stage failure: Task 4 in stage 19.0 failed 4 times, most recent failure: Lost task 4.3 in stage 19.0 (TID 70) (ip-172-31-83-211.ec2.internal executor 1): org.apache.spark.SparkException: This RDD lacks a SparkContext. It could happen in the following cases:
(1) RDD transformations and actions are NOT invoked by the driver, but inside of other transformations; for example, rdd1.map(x => rdd2.values.count() * x) is invalid because the values transformation and count action cannot be performed inside of the rdd1.map transformation. For more information, see SPARK-5063.
(2) When a Spark Streaming job recovers from checkpoint, this exception will be hit if a reference to an RDD not defined by the streaming job is used in DStream operations. For more information, See SPARK-13758.
	at org.apache.spark.errors.SparkCoreErrors$.rddLacksSparkContextError(SparkCoreErrors.scala:130)
	at org.apache.spark.rdd.RDD.sc(RDD.sca

In [ ]:
// Sentiment Analysis
bookSampleRdd
    .join